# Python scripting

Any python source code can be converted into a command line script by inspecting the global ```__name__``` variable.

In [15]:
def main():
    print('executing main program')

if __name__ == "__main__":
    main()

executing main program


By checking if ```__name__ == "__main__"```, we can tell if the document has been accessed from command line, and we can perform further operations. This is the method used by my command line programs. We will cover two methods here, ```click``` and ```argparse```.

## Click

Click supports decorator syntax, which allows the specification of command line arguments to happen outside of the main function.

In [16]:
%%python 
# the above line is just used for testing (not needed outside jupyter)

import click
import sys # this line needed for testing

@click.command()
@click.argument('integers', nargs = -1, type = int)
@click.option('--sum', 'accumulate', default=False, is_flag = True, 
              help='Whether to sum over inputs (default is max)')
def main(integers, accumulate):
    if accumulate:
        print(sum(integers))
    else:
        print(max(integers))
        

if __name__ == '__main__':
    sys.argv = ['', '3', '4', '5'] #for testing
    main() #click automatically calls sys.argv 

5


write the script to file, removing all testing lines

In [17]:
with open('cli/test_click_cli.py', 'w') as f:
    f.write('''
import click 

@click.command()
@click.argument('integers', nargs = -1, type = int)
@click.option('--sum', 'accumulate', default=False, is_flag = True, help='Whether to sum over inputs (default is max)')
def main(integers, accumulate):
    if accumulate:
        print(sum(integers))
    else:
        print(max(integers))
        

if __name__ == '__main__':
    main()
''')

In [18]:
!python cli/test_click_cli.py --help

Usage: test_click_cli.py [OPTIONS] [INTEGERS]...

Options:
  --sum   Whether to sum over inputs (default is max)
  --help  Show this message and exit.


In [19]:
!python cli/test_click_cli.py 3 4 5 --sum

12



To find out more on Click, visit [Click](https://click.palletsprojects.com/en/7.x/).

## Using Argparse
The most commonly-used argument parser is ```argparse```, which is included in python 2 and 3. Let's reproduce the above command-line interface using argparse.

This example borrows heavily from https://docs.python.org/2/library/argparse.html#adding-arguments

In [20]:
import argparse, sys

def main(argv=sys.argv[1:]):
    parser = argparse.ArgumentParser(description = "command line test")
    parser.add_argument('integers', metavar='N', type=int, nargs='+',
                    help='an integer for the accumulator')
    parser.add_argument('--sum', dest='accumulate', action='store_const',
                        const=sum, default=max,
                        help='sum the integers (default: find the max)')
    args = parser.parse_args(argv)
    print(args.accumulate(args.integers))

The ```main``` function will parse the positional input arguments, storing them in ```args.integers```. An optional ```--sum``` flag will be stored in ```args.accumulate```. 

For ```integers``` we have specified that the input type must be ```int``` and that we may accept a variable number of arguments ```nargs='+'```. 

For the ```--sum``` flag,  if the flag is set, the action to be taken is ```action=store_const```. In this case ```const=sum``` means that the ```sum``` function will be stored. However, if the flag is not set, the default behavior is to store the ```max``` function using ```default=max```.

In [21]:
main([str(i) for i in [3, 4, 5]]) # max(3, 4, 5)

5


In [22]:
main([str(i) for i in [3, 4, 5, '--sum']]) # 3+4+5

12


let's write our main method to a ```cli``` subdirectory as an executable python script.

In [23]:
argparse_cli = '''
import argparse,sys

def main(argv=sys.argv[1:]):
    parser = argparse.ArgumentParser(description = "command line test")
    parser.add_argument('integers', metavar='N', type=int, nargs='+',
                    help='an integer for the accumulator')
    parser.add_argument('--sum', dest='accumulate', action='store_const',
                        const=sum, default=max,
                        help='sum the integers (default: find the max)')
    args = parser.parse_args(argv)
    print(args.accumulate(args.integers))
    

if __name__ == '__main__':
    main(sys.argv[1:])
'''

In [24]:
with open('cli/test_argparse_cli.py', 'w') as f:
    f.write(argparse_cli)

Run the script and view the auto-generated help documentation

In [25]:
!python cli/test_argparse_cli.py -h

usage: test_argparse_cli.py [-h] [--sum] N [N ...]

command line test

positional arguments:
  N           an integer for the accumulator

optional arguments:
  -h, --help  show this help message and exit
  --sum       sum the integers (default: find the max)


test with and without ```--sum``` flag

In [26]:
!python cli/test_argparse_cli.py 3 4 5 8 --sum 

20


In [27]:
!python cli/test_argparse_cli.py 3 4 5 8

8


# Executable scripts

In the above examples, we invoke the script using

    python myscript.py <args> <options>

As long as we are running the script in a conda environment, this will work fine. But if you want to run the script as an executable outside an environment, there are two approaches:

## Using #/path/to/python (not recommended)

The easiest way to make a script executable is to hard-code a path to the appropriate python executable at the top of the script with a hash sign:

In [28]:
!which python #find out the path to your python interpreter

/Users/apembrok/miniconda2/envs/py101/bin/python


paste it as the first line at the top of your script

In [29]:
#/Users/apembrok/miniconda2/envs/python101/bin/python

if __name__ == '__main__':
    print('executing script')

executing script


Now write to file and use ```chmod``` to make the script executable.

One reason this is not recommended is that it makes the installation of the script non-portable. Other reasons can be found here:

http://click.palletsprojects.com/en/7.x/setuptools/?highlight=setuptools

## Using setuptools (recommended)

First we write the program to file as before

In [30]:
with open('cli/test_click_exe.py', 'w') as f:
    f.write('''
import click 

@click.command()
@click.argument('integers', nargs = -1, type = int)
@click.option('--sum', 'accumulate', default=False, is_flag = True, 
    help='Whether to sum over inputs (default is max)')
def main(integers, accumulate):
    if accumulate:
        print(sum(integers))
    else:
        print(max(integers))
''')

Note that we did not require the ```__main__``` check as before. That's because we will specify the ```entry_point``` for the executable in ```setup.py```.

In [31]:
with open('cli/setup.py','w') as f:
    f.write("""
from setuptools import setup

setup(
    name='test_click_exe',
    version='0.1',
    py_modules=['test_click_exe'],
    install_requires=[
        'Click',
    ],
    entry_points='''
        [console_scripts]
        test_click_exe=test_click_exe:main
    ''',
)
""")

In [32]:
!cat cli/setup.py


from setuptools import setup

setup(
    name='test_click_exe',
    version='0.1',
    py_modules=['test_click_exe'],
    install_requires=[
        'Click',
    ],
    entry_points='''
        [console_scripts]
        test_click_exe=test_click_exe:main
    ''',
)


## test your executable

Now, with your environment activated:

    (python101) cd cli
    (python101) pip install --editable .

This will place the executable in your environment's bin directory.

In [33]:
!which test_click_exe

In [34]:
!test_click_exe 3 4 5 --sum

/bin/sh: test_click_exe: command not found


Now we no longer need to have the environment activated to run it

    /Users/apembrok/miniconda2/envs/python101/bin/test_click_exe 3 3 3 --sum
    9

This means we can symlink the executable and make it available to other scripts.

In [35]:
!cat /Users/apembrok/miniconda2/envs/python101/bin/test_click_exe

#!/Users/apembrok/miniconda2/envs/python101/bin/python
# EASY-INSTALL-ENTRY-SCRIPT: 'test-click-exe','console_scripts','test_click_exe'
__requires__ = 'test-click-exe'
import re
import sys
from pkg_resources import load_entry_point

if __name__ == '__main__':
    sys.argv[0] = re.sub(r'(-script\.pyw?|\.exe)?$', '', sys.argv[0])
    sys.exit(
        load_entry_point('test-click-exe', 'console_scripts', 'test_click_exe')()
    )


To summarize, if you are building executable scripts:

* I highly recommend using click over argparse - it's easier to understand and you can build more complex appications later
* Use setup.py if you cannot guarantee that your script will be run with the appropriate python interpreter